In [8]:
!pip3 install textblob vaderSentiment transformers
!pip3 install torch torchvision torchaudio
!pip3 install tqdm



In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [6]:
import pandas as pd
file_path = "/content/drive/My Drive/nyt_articles_merged.csv"

dataframe = pd.read_csv(file_path, delimiter=",")
print(dataframe)
snippets = dataframe["snippet"]

                                                  title  \
0     An Ice Cream Mogul and His Fellow Burlingtonia...   
1     Bill Clinton, Back in Iowa, Refuses to Respond...   
2               Donald Trump Brings Gridlock to Vermont   
3     Ad Wars of 2016 Campaign Erupt in a Changing T...   
4     Ted Cruz’s Iowa Bus Tour: Using ‘Star Wars’ to...   
...                                                 ...   
3109  Trump Picks Key Figure in Project 2025 for Pow...   
3110  Pam Bondi, a $25,000 Donation and Trump Univer...   
3111                          Matt Gaetz and the D-word   
3112  Republicans Blocking Proposal to Narrow Part o...   
3113  Elon Musk Gets a Crash Course in How Trumpworl...   

                                                snippet  \
0     In a liberal enclave fiercely loyal to Bernie ...   
1     Bill Clinton returned to the campaign trail in...   
2     Donald J. Trump brought gridlock to Burlington...   
3     Candidates and the outside groups supporting t...

In [10]:
dataframe = dataframe.dropna(subset=['snippet'])  # Drop NaN in 'snippet' column

# Optionally, drop rows where 'snippet' is an empty string
dataframe = dataframe[dataframe['snippet'].str.strip() != '']
print(dataframe.shape)  # To verify the number of rows left


(3085, 6)


In [11]:
from transformers import pipeline
import torch
from tqdm import tqdm

sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

snippets_list = snippets.dropna().tolist()  # Drop NaN and convert to list
snippets_list = [str(snippet) for snippet in snippets_list]  # Ensure strings

results = []
for snippet in tqdm(snippets_list, desc="Analyzing Sentiment"):
    try:
        result = sentiment_pipeline(snippet)
        results.append(result[0])  # Append the first result (single example)
    except Exception as e:
        # Handle any errors (optional)
        results.append({"label": "ERROR", "score": 0.0})

# Add sentiment results to DataFrame
dataframe['sentiment_label'] = [res['label'] for res in results]
dataframe['sentiment_score'] = [res['score'] for res in results]

print(dataframe)

Analyzing Sentiment: 100%|██████████| 3085/3085 [10:12<00:00,  5.04it/s]

                                                  title  \
0     An Ice Cream Mogul and His Fellow Burlingtonia...   
1     Bill Clinton, Back in Iowa, Refuses to Respond...   
2               Donald Trump Brings Gridlock to Vermont   
3     Ad Wars of 2016 Campaign Erupt in a Changing T...   
4     Ted Cruz’s Iowa Bus Tour: Using ‘Star Wars’ to...   
...                                                 ...   
3109  Trump Picks Key Figure in Project 2025 for Pow...   
3110  Pam Bondi, a $25,000 Donation and Trump Univer...   
3111                          Matt Gaetz and the D-word   
3112  Republicans Blocking Proposal to Narrow Part o...   
3113  Elon Musk Gets a Crash Course in How Trumpworl...   

                                                snippet  \
0     In a liberal enclave fiercely loyal to Bernie ...   
1     Bill Clinton returned to the campaign trail in...   
2     Donald J. Trump brought gridlock to Burlington...   
3     Candidates and the outside groups supporting t...

In [12]:
save_path = "/content/drive/My Drive/nyt_articles_sentiment.csv"
dataframe.to_csv(save_path, index=False)